In [ ]:
import os
import shutil
import warnings
import csv
import yaml
import json
import torch

from PIL import Image
import pandas as pd
import numpy as np
from typing import Any, Sequence

from os import PathLike
from torch.utils.data import Dataset

from megadetector.detection.run_detector import load_detector, model_string_to_model_version
from megadetector.detection.run_detector_batch import process_images, write_results_to_file


from sklearn.model_selection import train_test_split

In [49]:
import random
import torch

from pathlib import Path
from PIL import Image

from torchvision.transforms import v2

from ba_dev.dataset import MammaliaData, MammaliaDataImage
from ba_dev.transform import ImagePipeline, BatchImagePipeline
from ba_dev.utils import load_config_yaml

paths = load_config_yaml('../path_config.yml')


### Running Tests

In [2]:
path_to_dataset = paths['dataset']
path_labelfiles = paths['test_labels']
path_to_detector_output = paths['md_output']
detector_model='mdv5a'
mode='train'

dataset = MammaliaDataImage(
    path_labelfiles=path_labelfiles,
    path_to_dataset=path_to_dataset,
    path_to_detector_output=path_to_detector_output,
    detector_model=detector_model,
    mode=mode,
)

/cfs/earth/scratch/kraftjul/BA_package/ba_dev/dataset.py:211: UserWarning: With the detection confidence of 0.25,
8 sequences had no detections and will be excluded.
Excluded sequences: [6000161, 6000163, 6000293, 6000530, 6000691, 6000372, 6000953, 6000186]
  warnings.warn(


In [3]:
pipline = ImagePipeline(
                path_to_dataset=path_to_dataset,
                pre_ops = [
                    ('to_rgb', {}),
                    ('crop_by_bb', {})
                ],
                transform = v2.Compose([
                                v2.ToImage(),
                                v2.ToDtype(torch.float32, scale=True),
                                v2.Resize((224, 224)),
                                ])
                )
                

In [4]:
row = dataset[77]

image = pipline(row['file_path'], row['bbox'])

print(image.shape)

torch.Size([3, 224, 224])


In [6]:
batch_pipline = BatchImagePipeline(
                path_to_dataset=path_to_dataset,
                num_workers=4,
                pre_ops = [
                    ('to_rgb', {}),
                    ('crop_by_bb', {}),
                ],
                transform = v2.Compose([
                                v2.ToImage(),
                                v2.ToDtype(torch.float32, scale=True),
                                v2.Resize((224, 224)),
                                ])
                )

In [7]:
list_of_paths = []
list_of_bboxes = []

samples = [random.randint(0, len(dataset)) for _ in range(100)]

for i in samples:
    row = dataset[i]

    list_of_paths.append(row['file_path'])
    list_of_bboxes.append(row['bbox'])

images = batch_pipline(list_of_paths, list_of_bboxes)

for image in images:
    print(image.shape)

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([

### Tests Feature Stats

In [28]:
import torch
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self):
        data = []
        for i in range(100):
            w = random.randint(10, 100)
            h = random.randint(10, 100)
            img = torch.randn(3, w, h)
            data.append(img)

        self.data = data

        
    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)
    

def collate_fn(batch):
    return batch

dataset = MyDataset()
loader = DataLoader(dataset, batch_size=10, num_workers=1, shuffle=False, collate_fn=collate_fn)


In [47]:
channel_sum = torch.zeros(3)
pixel_count = 0

for batch in loader:
    for img in batch:
        pixel_count += img.shape[1] * img.shape[2]
        for c in range(img.shape[0]):
            channel_sum[c] += img[c].sum()

mean = channel_sum / pixel_count

channel_diff_squared_sum = torch.zeros(3)
for batch in loader:
    for img in batch:
        img_centered_squared = (img - mean[:, None, None]) ** 2
        for c in range(img_centered_squared.shape[0]):
            channel_diff_squared_sum[c] += img_centered_squared[c].sum()

std = torch.sqrt(channel_diff_squared_sum / pixel_count)

print("Mean:", mean)
print("Std:", std)

Mean: tensor([-0.0027,  0.0015,  0.0001])
Std: tensor([0.9996, 0.9989, 1.0009])


In [51]:
import torch
from torch.utils.data import DataLoader, Dataset
from os import PathLike

class MammaliaDatasetFeatureStats(MammaliaDataImage):
    def __init__(
            self,
            path_labelfiles: str | PathLike,
            path_to_dataset: str | PathLike,
            path_to_detector_output: str | PathLike,
            detector_model: str | None = None,
            applied_detection_confidence: float = 0.25,
            available_detection_confidence: float = 0.25,
            random_seed: int = 55,
            test_size: float = 0.2,
            n_folds: int = 5,
            val_fold: int = 0,
            mode: str = 'init',
            ):
        super().__init__(
            path_labelfiles=path_labelfiles,
            path_to_dataset=path_to_dataset,
            path_to_detector_output=path_to_detector_output,
            detector_model=detector_model,
            applied_detection_confidence=applied_detection_confidence,
            available_detection_confidence=available_detection_confidence,
            random_seed=random_seed,
            test_size=test_size,
            n_folds=n_folds,
            val_fold=val_fold,
            mode=mode
        )

        self.image_pipline = ImagePipeline(
                path_to_dataset=self.path_to_dataset,
                pre_ops = [
                    ('to_rgb', {}),
                    ('crop_by_bb', {})
                ],
                transform = v2.Compose([
                                v2.ToImage(),
                                v2.ToDtype(torch.float32, scale=True),
                                ])
                )
        
    def __getitem__(self, index):
            row_index = self.row_map[index]
            row = self.ds.iloc[row_index]

            image_path = row['file_path']
            bbox = row['bbox']
            return self.image_pipline(image_path, bbox)
        

In [12]:
import torch
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(100, 3, 24, 24)
        
    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)
    

dataset = MyDataset()
loader = DataLoader(
    dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)


mean = 0.
std = 0.
nb_samples = 0.
for data in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

mean, std

(tensor([ 0.0017, -0.0062,  0.0045]), tensor([0.9961, 0.9990, 0.9995]))

In [13]:
import torch
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(100, 3, 24, 24)
        
    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)
    

dataset = MyDataset()
loader = DataLoader(
    dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)


channel_sum = 0.
channel_squared_sum = 0.
num_pixels = 0

for data in loader:
    # data shape: [B, C, H, W]
    data = data.float()
    B, C, H, W = data.shape
    pixels = data.view(B, C, -1)

    channel_sum += pixels.sum(dim=(0, 2))  # sum over B and all pixels
    channel_squared_sum += (pixels ** 2).sum(dim=(0, 2))
    num_pixels += B * H * W

mean = channel_sum / num_pixels
std = (channel_squared_sum / num_pixels - mean ** 2).sqrt()

mean, std

(tensor([ 0.0009, -0.0008,  0.0051]), tensor([1.0028, 1.0004, 0.9990]))